# DATA PREPROCESSING


In [39]:
# DATA PREPROCESSING
# MFCCs

import pandas as pd
import numpy as np
import tables  
import librosa
import librosa.display
import tensorflow


# time taken to read data
chunk = pd.read_excel('ESC-50-master/meta/miniesc50.xlsx',usecols=['filename','category'])
classes = chunk['category']
audiofiles = chunk['filename']

mfccFeatures = []
i = 0
for wavFile in audiofiles:
    data, samplingRate = librosa.load(('ESC-50-master/audio/' + wavFile), sr=22050)

    #Grab the mfcc features through the mfcc algorithm
    mfcc = librosa.feature.mfcc(data,sr=samplingRate,n_mfcc=40)
    mfccs_scaled_features = np.mean(mfcc.T,axis=0)


    mfccFeatures.append([mfccs_scaled_features,classes[i]])
    i += 1
    if (i == 200):
        break
pd.set_option('display.max_colwidth', None)
mfccFeatures = pd.DataFrame(mfccFeatures, columns=['feature','class_category'])


C:\Users\reals\AppData\Local\Temp\ipykernel_20712\3569254307.py:26: FutureWarning: Pass y=[-0.00965616 -0.01658902  0.00125793 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(data,sr=samplingRate,n_mfcc=40)
C:\Users\reals\AppData\Local\Temp\ipykernel_20712\3569254307.py:26: FutureWarning: Pass y=[-0.0058949  -0.00882469 -0.00778794 ...  0.          0.
  0.        ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(data,sr=samplingRate,n_mfcc=40)
C:\Users\reals\AppData\Local\Temp\ipykernel_20712\3569254307.py:26: FutureWarning: Pass y=[ 0.02252876  0.03561823  0.03206565 ... -0.07500877 -0.13603589
 -0.13165964] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(data,sr=samplingRate,n_mfcc=40)
C:\Users\reals\AppData\Local\Tem

# MODEL CREATION AND TESTING

In [4]:
# MODEL CREATION AND TESTING

import numpy as np
from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, ShuffleSplit
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.python.keras.optimizer_v2 import adam
from tensorflow.python.keras.callbacks import ModelCheckpoint
from sklearn import metrics
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from numpy import load
# dataset = pd.read_csv('/content/extractedMFCCFeaturesDataSet1.csv')
# X = np.array(mfccFeatures['feature'].tolist())
X = load('X.npy')
# Y = np.array(mfccFeatures['class_category'].tolist())
Y = load('Y.npy')

#Encode target labels with value between 0 and n-1 classes
Label_encoder = LabelEncoder()
yy=to_categorical((Label_encoder.fit_transform(Y)))

print(yy.shape)
print(X.shape)

#performing train test split on our data set.
X_train,X_test,Y_train,Y_test=train_test_split(X,yy,test_size=0.2,random_state=0)

num_labels=yy.shape[1]

model=Sequential()
#first layer
model.add(Dense(256,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

# To compile the model we need to define loss function which is categorical cross-entropy,
# accuracy metrics which is accuracy score, and an optimizer which is Adam.

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
model.summary()
# Calculate pre-training accuracy 
score = model.evaluate(X_test, Y_test, verbose=0)
accuracy = 100*score[1]

# TESTING BEFORE TRAINING
print(f'the accuracy {accuracy} before training')

# TRAINING
num_epochs = 200
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='./audio_classification.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(X_train, Y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, Y_test), callbacks=[checkpointer] ,verbose=1)

# EVALUATING AFTER TRAINING
score = model.evaluate(X_train, Y_train, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))

score = model.evaluate(X_test, Y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

# PREDICTION

ImportError: cannot import name 'Adam' from 'tensorflow.python.keras.optimizer_v2' (/home/ahmad0fahmy/Desktop/code/tensorflow/python/keras/optimizer_v2/__init__.py)

#PREDICTION


In [ ]:
filename="testSounds/Dog.wav"

#preprocess the audio file
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

#Reshape MFCC feature to 2-D array
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)

x_predict=model.predict(mfccs_scaled_features) 
predicted_label=np.argmax(x_predict,axis=1)

prediction_class = Label_encoder.inverse_transform(predicted_label) 
print(prediction_class)

NameError: name 'librosa' is not defined